# Assignment 1: Search Algorithms

## 🎯 Learning Objectives
- Implement and compare different search algorithms
- Understand the trade-offs between search strategies
- Apply search algorithms to classic AI problems

## Tasks
- Implement missing algorithms: DFS, UCS and A*
- Implement heuristics for sliding tiles problem (at least two)
- Implement complex vacuum cleaner
- Implement a labyrinth solver

In [2]:
# 📝 TODO: Change this to your name (use FIRSTNAME-SURNAME format)
STUDENT_NAME = 'Geethika Reddy Konda'


## 🔧 Setup (Run this first!)
This cell installs dependencies and sets up the API connection. **You only need to change your name below!**

In [3]:
# =====================================
# 🚫 DON'T MODIFY ANYTHING BELOW HERE
# =====================================

import requests
import json
import time
import copy
import datetime
from abc import abstractmethod
from collections import deque
from queue import PriorityQueue

# API Configuration
BASE_URL = 'https://ie-aireasoning-27700454894.europe-west1.run.app'

def get_problem(problem_type):
    """Get a problem instance from the server."""
    response = requests.get(f'{BASE_URL}/get-problem', params={
        'TOKEN': STUDENT_NAME,
        'problem': problem_type
    })
    if response.status_code != 200:
        print(f'❌ Error {response.status_code}: {response.text}')
        return None
    return response.json()

print(f"✅ Setup complete! Student: {STUDENT_NAME}")
print("🔗 Connected to AI Reasoning Server")

✅ Setup complete! Student: Geethika Reddy Konda
🔗 Connected to AI Reasoning Server


## 📚 Search Framework

These classes provide the foundation for all search algorithms. **You don't need to modify these**, but understanding them is crucial.

In [4]:
# =====================================
# 🚫 DON'T MODIFY ANYTHING BELOW HERE
# =====================================

class State:
    """Abstract base class for problem states.

    When implementing a State subclass, you MUST implement:
    - __str__(): for pretty printing
    - __eq__(): for state comparison
    - __hash__(): for using states in sets/dicts
    """
    pass

class Action:
    """Represents an action that transitions between states."""

    def __init__(self, name: str, resulting_state: State, cost: int = 1):
        self.name = name
        self.resulting_state = resulting_state
        self.cost = cost

    def __str__(self):
        return self.name

class SearchNode:
    """A node in the search tree."""

    def __init__(self, state: State, parent: 'SearchNode' = None,
                 action: Action = None, path_cost: int = 0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = 0 if parent is None else parent.path_cost + action.cost
        self.depth = 0 if parent is None else parent.depth + 1

class Problem:
    """Abstract base class for search problems."""

    @abstractmethod
    def initial_state(self) -> State:
        """Return the initial state."""
        pass

    @abstractmethod
    def is_goal(self, state: State) -> bool:
        """Check if state is a goal state."""
        pass

    @abstractmethod
    def actions(self, state: State) -> list[Action]:
        """Return available actions from given state."""
        pass

    def heuristic(self, state: State) -> int:
        """Heuristic function for informed search (optional).

        Make sure to switch based on the value of self.heuristic.
        """
        return 0

class SearchResult:
    """Container for search algorithm results."""

    def __init__(self, solution_node: SearchNode = None,
                 nodes_expanded: int = 0, execution_time: float = 0.0):
        self.solution_node = solution_node
        self.nodes_expanded = nodes_expanded
        self.execution_time = execution_time
        self.found_solution = solution_node is not None

    def solution_path(self) -> list[str]:
        """Extract the sequence of actions from initial state to goal."""
        if not self.found_solution:
            return []

        path = []
        node = self.solution_node
        while node.parent is not None:
            path.insert(0, node.action.name)
            node = node.parent
        return path

    def __str__(self):
        if not self.found_solution:
            return f"❌ No solution found\n⏱️  Time: {self.execution_time:.4f}s\n🔍 Nodes expanded: {self.nodes_expanded}"

        path = self.solution_path()
        return (
            f"✅ Solution found!\n"
            f"📊 Cost: {self.solution_node.path_cost}\n"
            f"📏 Steps: {len(path)}\n"
            f"⏱️  Time: {self.execution_time:.4f}s\n"
            f"🔍 Nodes expanded: {self.nodes_expanded}\n"
            f"🛤️  Path: {' → '.join(path) if len(path) <= 20 else ' → '.join(path[:20]) + '...'}"
        )

def expand_node(problem: Problem, node: SearchNode) -> list[SearchNode]:
    """Generate child nodes from a parent node."""
    children = []
    for action in problem.actions(node.state):
        child = SearchNode(
            state=action.resulting_state,
            parent=node,
            action=action
        )
        children.append(child)
    return children

print("📚 Search framework loaded!")

📚 Search framework loaded!


## Breadth first search

This is a working implementation of BFS. **You don't need to modify it**, but understanding it is useful to implement the other algotirthms.

In [5]:
# =====================================
# 🚫 DON'T MODIFY ANYTHING BELOW HERE
# =====================================

def breadth_first_search(problem: Problem) -> SearchResult:
    """🔍 Breadth-First Search implementation.

    ✅ This is implemented as an example to help you understand the pattern.
    Study this implementation, then use it as a template for DFS and UCS!

    Key BFS characteristics:
    - Uses a FIFO queue (deque with popleft/append)
    - Explores nodes level by level
    - Guarantees optimal solution for unweighted graphs
    - Complete: will find solution if one exists
    """
    start_time = time.perf_counter()

    # Initialize with the initial state
    initial_node = SearchNode(problem.initial_state())

    # Early goal test - return immediately if we start at the goal
    if problem.is_goal(initial_node.state):
        return SearchResult(initial_node, 0, time.perf_counter() - start_time)

    # BFS uses a FIFO queue for the frontier
    frontier = deque([initial_node])
    visited = {initial_node.state}  # Track visited states to avoid cycles
    nodes_expanded = 0

    while frontier:
        # BFS: take from FRONT of queue (FIFO - First In, First Out)
        current_node = frontier.popleft()
        nodes_expanded += 1

        # Generate all possible next states
        for child in expand_node(problem, current_node):
            if child.state not in visited:
                # Goal test BEFORE adding to frontier (early goal test)
                if problem.is_goal(child.state):
                    return SearchResult(child, nodes_expanded, time.perf_counter() - start_time)

                # Mark as visited and add to frontier
                visited.add(child.state)
                frontier.append(child)  # Add to BACK of queue

    # No solution found
    return SearchResult(None, nodes_expanded, time.perf_counter() - start_time)

print("🔍 BFS implementation loaded (complete example)!")

🔍 BFS implementation loaded (complete example)!


## Depth first search

This algorithm is for you to implement! Make user you use the given classes before, so that you can run the same problems with the different versions of search. Use BFS as inspiration, but remember: **you can't just copy BFS as things like early goal test don't work in DFS**

In [6]:
def depth_first_search(problem: Problem) -> SearchResult:
    """🔍 Depth-First Search implementation.

    Key DFS characteristics:
    - Uses a LIFO stack (regular list with append/pop)
    - Explores deeply before exploring siblings
    - May not find optimal solution
    - Can get stuck in infinite loops without cycle detection
    """
    start_time = time.perf_counter()

    # Initialize with the initial state
    initial_node = SearchNode(problem.initial_state())

    # Early goal test
    if problem.is_goal(initial_node.state):
        return SearchResult(initial_node, 0, time.perf_counter() - start_time)

    # DFS uses a LIFO stack for the frontier
    frontier = [initial_node]  # Regular list as stack
    visited = {initial_node.state}  # Track visited states
    nodes_expanded = 0

    while frontier:
        # DFS: take from the END of list (LIFO - Last In, First Out)
        current_node = frontier.pop()
        nodes_expanded += 1

        # Expand node (generate successors)
        for child in expand_node(problem, current_node):
            if child.state not in visited:
                # Early goal test
                if problem.is_goal(child.state):
                    return SearchResult(child, nodes_expanded, time.perf_counter() - start_time)

                # Mark visited and add to stack
                visited.add(child.state)
                frontier.append(child)  # Add to END for LIFO

    # No solution found
    return SearchResult(None, nodes_expanded, time.perf_counter() - start_time)

print("🔍 DFS implementation loaded (using LIFO stack)!")


🔍 DFS implementation loaded (using LIFO stack)!


## Uniform cost search (or Dijsktra)

For you to implement.

In [7]:
import heapq
import time

def uniform_cost_search(problem: Problem) -> SearchResult:
    """🔍 Uniform Cost Search (UCS)

    Characteristics:
    - Expands the lowest-cost node first (like Dijkstra’s algorithm)
    - Guarantees optimal solution for positive costs
    - Complete if all step costs are positive
    """

    start_time = time.perf_counter()

    # Initialize start node
    initial_state = problem.initial_state()
    initial_node = SearchNode(initial_state, path_cost=0)

    # Early goal test
    if problem.is_goal(initial_state):
        return SearchResult(initial_node, 0, time.perf_counter() - start_time)

    # Priority queue (min-heap)
    frontier = []
    counter = 0
    heapq.heappush(frontier, (0, counter, initial_node))

    # Best known cost for each visited state
    best_cost = {initial_state: 0}

    nodes_expanded = 0

    while frontier:
        cost, _, current_node = heapq.heappop(frontier)

        # Skip outdated entries (when a cheaper path was found)
        if cost > best_cost.get(current_node.state, float('inf')):
            continue

        # Goal test on pop
        if problem.is_goal(current_node.state):
            return SearchResult(current_node, nodes_expanded, time.perf_counter() - start_time)

        nodes_expanded += 1

        # Expand node
        for child in expand_node(problem, current_node):
            child_cost = child.path_cost  # already includes parent cost + action cost

            # If this path to child is cheaper than any known
            if child_cost < best_cost.get(child.state, float('inf')):
                best_cost[child.state] = child_cost
                counter += 1
                heapq.heappush(frontier, (child_cost, counter, child))

    # No solution found
    return SearchResult(None, nodes_expanded, time.perf_counter() - start_time)


## A* search

For you to implement.

In [8]:
from queue import PriorityQueue
import time

def a_star_search(problem: Problem) -> SearchResult:
    """🔍 A* Search implementation."""
    start_time = time.perf_counter()

    initial_state = problem.initial_state()
    initial_node = SearchNode(initial_state, path_cost=0)

    # Early goal test
    if problem.is_goal(initial_state):
        return SearchResult(initial_node, 0, time.perf_counter() - start_time)

    # Safe heuristic handling
    if callable(problem.heuristic):
        h_start = problem.heuristic(initial_state)
    else:
        h_start = 0

    frontier = PriorityQueue()
    counter = 0
    frontier.put((h_start, counter, initial_node))

    visited = {initial_state: 0}
    nodes_expanded = 0

    while not frontier.empty():
        f_cost, _, current_node = frontier.get()
        nodes_expanded += 1

        if problem.is_goal(current_node.state):
            return SearchResult(current_node, nodes_expanded, time.perf_counter() - start_time)

        for child in expand_node(problem, current_node):
            g = child.path_cost

            # Safe heuristic usage again
            if callable(problem.heuristic):
                h = problem.heuristic(child.state)
            else:
                h = 0

            f = g + h

            if (child.state not in visited) or (g < visited[child.state]):
                visited[child.state] = g
                counter += 1
                frontier.put((f, counter, child))

    return SearchResult(None, nodes_expanded, time.perf_counter() - start_time)


## Helper code

**There is no need to modify this code**. This code is used to compare algorithms easily.

In [9]:
# =====================================
# 🚫 DON'T MODIFY ANYTHING BELOW HERE
# =====================================

def compare_algorithms(problem: Problem, problem_name: str):
    """Compare different search algorithms on the same problem."""

    algorithms = [
        ("Breadth-First Search", breadth_first_search),
        ("Depth-First Search", depth_first_search),
        ("Uniform Cost Search", uniform_cost_search),
        ("A* Search", a_star_search),
    ]

    print(f"🏁 Comparing algorithms on: {problem_name}\n")
    print(f"{'Algorithm':<20} {'Found?':<8} {'Cost':<6} {'Nodes':<8} {'Time (s)':<10}")
    print("-" * 60)

    for name, algorithm in algorithms:
        try:
            result = algorithm(problem)
            found = "✅ Yes" if result.found_solution else "❌ No"
            cost = result.solution_node.path_cost if result.found_solution else "N/A"
            nodes = result.nodes_expanded
            time_taken = f"{result.execution_time:.4f}"

            print(f"{name:<20} {found:<8} {cost:<6} {nodes:<8} {time_taken:<10}")
        except Exception as e:
            print(f"{name:<20} {'❌ Error':<8} {'N/A':<6} {'N/A':<8} {'N/A':<10}")
            print(f"  Error: {e}")

print("📊 Algorithm comparison function loaded!")

📊 Algorithm comparison function loaded!


## 🧹 Simple Vacuum Problem

Now let's test our search algorithms on a classic AI problem: the **vacuum cleaner world**.

**Problem Description:**
- **Environment**: Two rooms (Room 0 and Room 1)
- **Robot**: Starts in Room 0
- **Goal**: Clean all dirt from both rooms
- **Actions**:
  - 🧹 **Suck**: Clean dirt in current room
  - ⬅️ **Left**: Move to Room 0 (if not already there)
  - ➡️ **Right**: Move to Room 1 (if not already there)

This is a perfect problem for testing search algorithms because:
- Small state space (easy to understand)
- Clear goal condition (no dirt anywhere)
- Multiple possible solution paths
- Good for comparing algorithm performance

In [63]:
class VacuumState(State):
    """State for the vacuum cleaner world."""

    def __init__(self, dirt_locations: list[bool], vacuum_position: int):
        self.dirt = dirt_locations.copy()
        self.position = vacuum_position

    def __str__(self):
        room_display = []
        for i, has_dirt in enumerate(self.dirt):
            if i == self.position:
                room_display.append('🤖' if not has_dirt else '🤖💨')
            else:
                room_display.append('💨' if has_dirt else '✨')
        return ' | '.join(room_display)

    def __eq__(self, other):
        return self.dirt == other.dirt and self.position == other.position

    def __hash__(self):
        return hash((tuple(self.dirt), self.position))

class VacuumProblem(Problem):
    """Two-room vacuum cleaner problem."""

    def __init__(self, heuristic: str = "none"):
        self.heuristic = heuristic

    def initial_state(self) -> State:
        return VacuumState([True, True], 0)  # Both rooms dirty, start at room 0

    def is_goal(self, state: VacuumState) -> bool:
        return not any(state.dirt)  # No dirt anywhere

    def actions(self, state: VacuumState) -> list[Action]:
        actions = []

        # Suck action
        new_dirt = state.dirt.copy()
        new_dirt[state.position] = False
        actions.append(Action("🧹 Suck", VacuumState(new_dirt, state.position)))

        # Move actions
        if state.position > 0:
            actions.append(Action("⬅️ Left", VacuumState(state.dirt, state.position - 1)))
        if state.position < len(state.dirt) - 1:
            actions.append(Action("➡️ Right", VacuumState(state.dirt, state.position + 1)))

        return actions

    def heuristic(self, state: State) -> int:
        """Heuristic function for informed search (optional).

        Make sure to switch based on the value of self.heuristic.
        """
        if self.heuristic == "simple":
            return sum(state.dirt)  # Number of dirty rooms
        else:
            return 0

# 🏁 Compare all algorithms on the simple vacuum problem
print("📊 Testing all algorithms on Simple Vacuum Problem:")
compare_algorithms(VacuumProblem(heuristic=None), "Simple Vacuum Cleaner")

# Show the problem visually
print(f"\n🎯 Initial state: {VacuumProblem(heuristic=None).initial_state()}")

# Show BFS solution as an example
print(f"\n🔍 BFS Solution:")
result = breadth_first_search(VacuumProblem(heuristic=None))
print(result)

📊 Testing all algorithms on Simple Vacuum Problem:
🏁 Comparing algorithms on: Simple Vacuum Cleaner

Algorithm            Found?   Cost   Nodes    Time (s)  
------------------------------------------------------------
Breadth-First Search ✅ Yes    3      4        0.0001    
Depth-First Search   ✅ Yes    4      4        0.0000    
Uniform Cost Search  ✅ Yes    3      5        0.0000    
A* Search            ✅ Yes    3      6        0.0001    

🎯 Initial state: 🤖💨 | 💨

🔍 BFS Solution:
✅ Solution found!
📊 Cost: 3
📏 Steps: 3
⏱️  Time: 0.0000s
🔍 Nodes expanded: 4
🛤️  Path: 🧹 Suck → ➡️ Right → 🧹 Suck


## 🏗️ Tower of Hanoi Problem

Let's explore another classic problem: the **Tower of Hanoi puzzle**.

**Problem Description:**
- **Setup**: Three pegs (A, B, C) with disks of different sizes
- **Initial**: All disks on peg A, largest at bottom, smallest at top
- **Goal**: Move all disks to peg C
- **Rules**:
  - Move one disk at a time
  - Can only move the top disk from a peg
  - Never place a larger disk on a smaller disk

This problem demonstrates:
- **Exponential state space**: 3ⁿ possible configurations for n disks
- **Action constraints**: Not all moves are legal at each state
- **Optimal solutions exist**: Minimum 2ⁿ-1 moves needed
- **Good for testing**: Clear goal, deterministic, well-understood

In [68]:
class HanoiState(State):
    """State for the Tower of Hanoi problem."""

    def __init__(self, pegs: list[list[int]]):
        # Each peg is a list of disk sizes, bottom to top
        # e.g., [[3,2,1], [], []] means all disks on peg A
        self.pegs = [peg.copy() for peg in pegs]

    def __str__(self):
        """Visual representation of the towers."""
        max_height = max(len(peg) for peg in self.pegs)
        lines = []

        # Build from top to bottom
        for level in range(max_height - 1, -1, -1):
            line = ""
            for peg_idx, peg in enumerate(self.pegs):
                if level < len(peg):
                    disk_size = peg[level]
                    disk_repr = "●" * disk_size
                else:
                    disk_repr = "|"
                line += f"{disk_repr:^7}"
            lines.append(line)

        # Add base and labels
        lines.append("═══════" * 3)
        lines.append("   A      B      C   ")
        return "\n".join(lines)

    def __eq__(self, other):
        return self.pegs == other.pegs

    def __hash__(self):
        return hash(tuple(tuple(peg) for peg in self.pegs))

class HanoiProblem(Problem):
    """Tower of Hanoi problem with configurable number of disks."""

    def __init__(self, heuristic: str = "none", num_disks: int = 3):
        self.heuristic = heuristic
        self.num_disks = num_disks
        # Initial state: all disks on peg A (largest to smallest)
        self.initial_pegs = [list(range(num_disks, 0, -1)), [], []]
        # Goal state: all disks on peg C
        self.goal_pegs = [[], [], list(range(num_disks, 0, -1))]

    def initial_state(self) -> State:
        return HanoiState(self.initial_pegs)

    def is_goal(self, state: HanoiState) -> bool:
        return state.pegs == self.goal_pegs

    def actions(self, state: HanoiState) -> list[Action]:
        actions = []
        peg_names = ['A', 'B', 'C']

        # Try moving from each peg to each other peg
        for from_peg in range(3):
            if state.pegs[from_peg]:  # Peg has disks
                disk = state.pegs[from_peg][-1]  # Top disk

                for to_peg in range(3):
                    if from_peg != to_peg:
                        # Check if move is legal (smaller on larger)
                        if not state.pegs[to_peg] or disk < state.pegs[to_peg][-1]:
                            # Create new state with disk moved
                            new_pegs = [peg.copy() for peg in state.pegs]
                            new_pegs[from_peg].pop()  # Remove from source
                            new_pegs[to_peg].append(disk)  # Add to destination

                            action_name = f"Move disk {disk}: {peg_names[from_peg]} → {peg_names[to_peg]}"
                            actions.append(Action(action_name, HanoiState(new_pegs)))

        return actions

    def heuristic(self, state: HanoiState) -> int:
        """Heuristic function for informed search (optional).

        Make sure to switch based on the value of self.heuristic.
        """
        if self.heuristic == "simple":
            return len(state.pegs[0]) + len(state.pegs[1]) # Count disks not on goal peg
        else:
            return 0

# Test the Tower of Hanoi problem
print("🏗️ Testing Tower of Hanoi Problem (3 disks):")
hanoi_problem = HanoiProblem(heuristic=None, num_disks=3)

print(f"\n🎯 Initial state:")
print(hanoi_problem.initial_state())

print(f"\n🎯 Goal state:")
goal_state = HanoiState(hanoi_problem.goal_pegs)
print(goal_state)

print(f"\n🔍 BFS Solution:")
result = breadth_first_search(hanoi_problem)
print(result)

if result.found_solution:
    print(f"\n🛤️ Solution path:")
    for i, action in enumerate(result.solution_path(), 1):
        print(f"{i:2d}. {action}")

# Compare algorithms on Hanoi problem
print(f"\n📊 Comparing algorithms on Tower of Hanoi:")
compare_algorithms(hanoi_problem, "Tower of Hanoi (3 disks)")


🏗️ Testing Tower of Hanoi Problem (3 disks):

🎯 Initial state:
   ●      |      |   
  ●●      |      |   
  ●●●     |      |   
═════════════════════
   A      B      C   

🎯 Goal state:
   |      |      ●   
   |      |     ●●   
   |      |     ●●●  
═════════════════════
   A      B      C   

🔍 BFS Solution:
✅ Solution found!
📊 Cost: 7
📏 Steps: 7
⏱️  Time: 0.0003s
🔍 Nodes expanded: 18
🛤️  Path: Move disk 1: A → C → Move disk 2: A → B → Move disk 1: C → B → Move disk 3: A → C → Move disk 1: B → A → Move disk 2: B → C → Move disk 1: A → C

🛤️ Solution path:
 1. Move disk 1: A → C
 2. Move disk 2: A → B
 3. Move disk 1: C → B
 4. Move disk 3: A → C
 5. Move disk 1: B → A
 6. Move disk 2: B → C
 7. Move disk 1: A → C

📊 Comparing algorithms on Tower of Hanoi:
🏁 Comparing algorithms on: Tower of Hanoi (3 disks)

Algorithm            Found?   Cost   Nodes    Time (s)  
------------------------------------------------------------
Breadth-First Search ✅ Yes    7      18       0.0002    
D

## 🧩 Sliding Tile Puzzle (8-Puzzle)

Our third example is the classic **8-puzzle** (3×3 sliding tile puzzle).

**Problem Description:**
- **Grid**: 3×3 with tiles numbered 1-8 and one empty space (0)
- **Goal**: Arrange tiles in order: `[[1,2,3], [4,5,6], [7,8,0]]`
- **Actions**: Slide a tile into the empty space (Up/Down/Left/Right)
- **Constraints**: Can only move tiles adjacent to the empty space

This problem is excellent for testing because:
- **Large state space**: 9! = 362,880 possible configurations
- **Variable difficulty**: Some shuffles are harder than others  
- **Heuristic-friendly**: Great for testing A* with different heuristics
- **Well-studied**: Known optimal solutions and heuristic functions

**Note**: We've intentionally **not provided a heuristic** for this problem - implementing good heuristics (like Manhattan distance) is part of your A* learning experience!

In [69]:
class SlidingPuzzleState(State):
    """State for the 3x3 sliding tile puzzle."""

    def __init__(self, grid: list[list[int]]):
        self.grid = [row.copy() for row in grid]
        # Find empty space (0) position
        self.empty_pos = None
        for i in range(3):
            for j in range(3):
                if self.grid[i][j] == 0:
                    self.empty_pos = (i, j)
                    break

    def __str__(self):
        """Visual representation of the puzzle."""
        lines = []
        lines.append("┌─────────┐")
        for row in self.grid:
            line = "│"
            for tile in row:
                if tile == 0:
                    line += "   "  # Empty space
                else:
                    line += f" {tile} "
            line += "│"
            lines.append(line)
        lines.append("└─────────┘")
        return "\n".join(lines)

    def __eq__(self, other):
        return self.grid == other.grid

    def __hash__(self):
        return hash(tuple(tuple(row) for row in self.grid))

class SlidingPuzzleProblem(Problem):
    """3x3 sliding tile puzzle (8-puzzle)."""

    def __init__(self, initial_grid: list[list[int]] = None, heuristic: str = "none"):
        self.heuristic_type = heuristic
        if initial_grid is None:
            # Get a puzzle from the server
            print("🌐 Fetching puzzle from server...")
            puzzle_data = get_problem('sliding_tiles')
            if puzzle_data and 'initial-state' in puzzle_data:
                # Convert server format \"4,3,5,7,1,6,8,2,0\" to 3x3 grid
                flat_state = [int(x) for x in puzzle_data['initial-state'].split(',')]
                self.initial_grid = [
                    flat_state[0:3],
                    flat_state[3:6],
                    flat_state[6:9]]
            else:
                print("⚠️  Server unavailable, using default puzzle...")
                # Fallback to a moderately difficult starting configuration
                self.initial_grid = [
                    [2, 8, 3],
                    [1, 6, 4],
                    [7, 0, 5]
                ]
        else:
            self.initial_grid = initial_grid

        # Goal state: tiles in order with empty space at bottom right
        self.goal_grid = [
            [1, 2, 3],
            [4, 5, 6],
            [7, 8, 0]
        ]

    def initial_state(self) -> State:
        return SlidingPuzzleState(self.initial_grid)

    def is_goal(self, state: SlidingPuzzleState) -> bool:
        return state.grid == self.goal_grid

    def actions(self, state: SlidingPuzzleState) -> list[Action]:
        actions = []
        empty_row, empty_col = state.empty_pos

        # Possible moves: Up, Down, Left, Right (moving tiles into empty space)
        moves = [
            ("Up", -1, 0),      # Move tile from above down
            ("Down", 1, 0),     # Move tile from below up
            ("Left", 0, -1),    # Move tile from left right
            ("Right", 0, 1)     # Move tile from right left
        ]

        for direction, dr, dc in moves:
            # Position of tile to move INTO the empty space
            tile_row, tile_col = empty_row + dr, empty_col + dc

            # Check if move is within bounds
            if 0 <= tile_row < 3 and 0 <= tile_col < 3:
                # Create new state with tile moved
                new_grid = [row.copy() for row in state.grid]
                tile_number = new_grid[tile_row][tile_col]

                # Swap empty space with tile
                new_grid[empty_row][empty_col] = tile_number
                new_grid[tile_row][tile_col] = 0

                action_name = f"Move {tile_number} {direction}"
                actions.append(Action(action_name, SlidingPuzzleState(new_grid)))

        return actions

    def heuristic(self, state: SlidingPuzzleState) -> int:
        """Heuristic function for A* search.

        Ideas for heuristics:
        1. Misplaced tiles: Count how many tiles are not in goal position
        2. Manhattan distance: Sum of distances each tile is from its goal position
        3. Linear conflicts: Additional cost for tiles that block each other
        """

        if self.heuristic_type == "misplaced":
            # Count tiles not in their goal position (excluding empty tile)
            count = 0
            for i in range(3):
                for j in range(3):
                    if state.grid[i][j] != 0 and state.grid[i][j] != self.goal_grid[i][j]:
                        count += 1
            return count

        elif self.heuristic_type == "manhattan":
            # Sum of Manhattan distances for each tile to its goal position
            distance = 0
            for i in range(3):
                for j in range(3):
                    tile = state.grid[i][j]
                    if tile != 0:  # Skip empty tile
                        # Find goal position of this tile
                        goal_row = (tile - 1) // 3
                        goal_col = (tile - 1) % 3
                        # Add Manhattan distance
                        distance += abs(i - goal_row) + abs(j - goal_col)
            return distance

        elif self.heuristic_type == "linear_conflict":
            # Manhattan distance + linear conflicts
            manhattan = 0
            conflicts = 0

            # Calculate Manhattan distance
            for i in range(3):
                for j in range(3):
                    tile = state.grid[i][j]
                    if tile != 0:
                        goal_row = (tile - 1) // 3
                        goal_col = (tile - 1) % 3
                        manhattan += abs(i - goal_row) + abs(j - goal_col)

            # Count linear conflicts in rows
            for i in range(3):
                for j in range(3):
                    tile_j = state.grid[i][j]
                    if tile_j != 0 and (tile_j - 1) // 3 == i:  # Tile belongs in this row
                        for k in range(j + 1, 3):
                            tile_k = state.grid[i][k]
                            if tile_k != 0 and (tile_k - 1) // 3 == i:  # Also belongs in this row
                                # Check if they're in wrong order
                                goal_col_j = (tile_j - 1) % 3
                                goal_col_k = (tile_k - 1) % 3
                                if goal_col_j > goal_col_k:
                                    conflicts += 2  # Each conflict costs 2 moves

            # Count linear conflicts in columns
            for j in range(3):
                for i in range(3):
                    tile_i = state.grid[i][j]
                    if tile_i != 0 and (tile_i - 1) % 3 == j:  # Tile belongs in this column
                        for k in range(i + 1, 3):
                            tile_k = state.grid[k][j]
                            if tile_k != 0 and (tile_k - 1) % 3 == j:  # Also belongs in this column
                                # Check if they're in wrong order
                                goal_row_i = (tile_i - 1) // 3
                                goal_row_k = (tile_k - 1) // 3
                                if goal_row_i > goal_row_k:
                                    conflicts += 2

            return manhattan + conflicts

        else:  # "none" or any other value
            # No heuristic (makes A* behave like UCS)
            return 0
# Test the sliding puzzle
print("🧩 Testing Sliding Tile Puzzle (8-puzzle):")
puzzle = SlidingPuzzleProblem()

print(f"\n🎯 Initial state:")
print(puzzle.initial_state())

print(f"\n🎯 Goal state:")
goal_state = SlidingPuzzleState(puzzle.goal_grid)
print(goal_state)

print(f"\n🔍 BFS Solution (this may take a moment for harder puzzles...):")
result = breadth_first_search(puzzle)
print(result)

if result.found_solution and len(result.solution_path()) <= 15:
    print(f"\n🛤️ Solution path:")
    for i, action in enumerate(result.solution_path(), 1):
        print(f"{i:2d}. {action}")

# Compare different heuristics
print(f"\n{'='*70}")
print("📊 COMPARING HEURISTICS")
print(f"{'='*70}")

initial_grid = puzzle.initial_grid

print("\n1️⃣ A* with NO heuristic:")
puzzle1 = SlidingPuzzleProblem(initial_grid=initial_grid, heuristic="none")
result1 = a_star_search(puzzle1)  # remove puzzle1.heuristic
print(result1)

print("\n2️⃣ A* with MISPLACED TILES heuristic:")
puzzle2 = SlidingPuzzleProblem(initial_grid=initial_grid, heuristic="misplaced")
result2 = a_star_search(puzzle2)  # remove puzzle2.heuristic
print(result2)

print("\n3️⃣ A* with MANHATTAN DISTANCE heuristic:")
puzzle3 = SlidingPuzzleProblem(initial_grid=initial_grid, heuristic="manhattan")
result3 = a_star_search(puzzle3)  # remove puzzle3.heuristic
print(result3)

print("\n4️⃣ A* with LINEAR CONFLICT heuristic:")
puzzle4 = SlidingPuzzleProblem(initial_grid=initial_grid, heuristic="linear_conflict")
result4 = a_star_search(puzzle4)  # remove puzzle4.heuristic
print(result4)

# Compare algorithms on sliding puzzle
print(f"\n📊 Comparing algorithms on Sliding Puzzle:")
print("⚠️  Note: Without a good heuristic, A* will behave like UCS")
print("💡 Tip: Implement the heuristic function to see A* shine!")
compare_algorithms(puzzle, "Sliding Puzzle (8-puzzle)")


🧩 Testing Sliding Tile Puzzle (8-puzzle):
🌐 Fetching puzzle from server...

🎯 Initial state:
┌─────────┐
│ 1  3  6 │
│ 8  2  7 │
│ 4  5    │
└─────────┘

🎯 Goal state:
┌─────────┐
│ 1  2  3 │
│ 4  5  6 │
│ 7  8    │
└─────────┘

🔍 BFS Solution (this may take a moment for harder puzzles...):
✅ Solution found!
📊 Cost: 14
📏 Steps: 14
⏱️  Time: 0.0462s
🔍 Nodes expanded: 1896
🛤️  Path: Move 7 Up → Move 6 Up → Move 3 Left → Move 2 Down → Move 5 Down → Move 7 Right → Move 6 Up → Move 5 Left → Move 8 Left → Move 4 Down → Move 7 Right → Move 8 Up → Move 5 Right → Move 6 Down

🛤️ Solution path:
 1. Move 7 Up
 2. Move 6 Up
 3. Move 3 Left
 4. Move 2 Down
 5. Move 5 Down
 6. Move 7 Right
 7. Move 6 Up
 8. Move 5 Left
 9. Move 8 Left
10. Move 4 Down
11. Move 7 Right
12. Move 8 Up
13. Move 5 Right
14. Move 6 Down

📊 COMPARING HEURISTICS

1️⃣ A* with NO heuristic:
✅ Solution found!
📊 Cost: 14
📏 Steps: 14
⏱️  Time: 0.0637s
🔍 Nodes expanded: 2961
🛤️  Path: Move 7 Up → Move 6 Up → Move 3 Left → Move 2 D

## Advanced Vacuum Problem!

**Challenge**: Extend the vacuum cleaner to handle a more complex scenario:

🤖 **Robot Vacuum 2.0 Specifications:**
- **Grid**: 4×4 rooms (16 rooms total)
- **Battery**: Starts with 8 units of charge
- **Energy costs**: Moving = 1 unit, Sucking = 1 unit
- **Charger**: Located at position (0,0) - the starting position
- **Charging**: When at (0,0), can charge back to full (8 units)
- **Goal**: Clean all rooms and return to charger

This problem introduces **resource management** - the robot must plan efficiently to avoid running out of battery!

In [17]:
class VacuumAction:
    """Action class compatible with the search framework."""
    def __init__(self, name: str, resulting_state: State, cost: int = 1):
        self.name = name
        self.resulting_state = resulting_state
        self.result = resulting_state  # For compatibility
        self.cost = cost  # <-- needed by search algorithms

    def __str__(self):
        return self.name



class AdvancedVacuumState(State):
    """State for the advanced 4x4 vacuum cleaner with battery management."""

    def __init__(self, dirt_grid: list[list[bool]], position: tuple[int, int], battery: int):
        self.dirt_grid = [row.copy() for row in dirt_grid]
        self.position = position
        self.battery = battery

    def __str__(self):
        """Create a visual representation of the current state."""
        lines = []
        lines.append(f"🔋 Battery: {self.battery}/15")
        lines.append("┌─────────────────┐")

        for i in range(4):
            line = "│"
            for j in range(4):
                if self.position == (i, j):
                    line += " 🤖"
                elif (i, j) == (0, 0):
                    if self.dirt_grid[i][j]:
                        line += " 🔌💨"
                    else:
                        line += " 🔌"
                elif self.dirt_grid[i][j]:
                    line += " 💨"
                else:
                    line += " ✨"
            line += " │"
            lines.append(line)

        lines.append("└─────────────────┘")
        return "\n".join(lines)

    def __eq__(self, other):
        return (self.dirt_grid == other.dirt_grid and
                self.position == other.position and
                self.battery == other.battery)

    def __hash__(self):
        return hash((tuple(tuple(row) for row in self.dirt_grid),
                    self.position,
                    self.battery))


class AdvancedVacuumProblem(Problem):
    """Advanced vacuum cleaner problem with battery management.

    🤖 Robot Vacuum 2.0 Specifications:
    - Action cost (time): Moving = 1, Sucking = 1, Charging = 10
    - Grid: 4×4 rooms (16 rooms total)
    - Battery: Starts with 15 units of charge
    - Energy costs: Moving = -1 unit, Sucking = -1 unit, Charging = 10 units
    - Charger: Located at position (0,0) - the starting position
    - Charging: When at (0,0), can charge back to full (15 units)
    - Goal: Clean all rooms and return to charger
    """

    def __init__(self, heuristic: str = "none"):
        self.heuristic_type = heuristic
        self.grid_size = 4
        self.charger_position = (0, 0)
        self.max_battery = 15

    def initial_state(self) -> State:
        # All rooms start dirty (True = dirty)
        dirt_grid = [[True for _ in range(4)] for _ in range(4)]
        # Robot starts at charger with full battery
        return AdvancedVacuumState(dirt_grid, self.charger_position, self.max_battery)

    def is_goal(self, state: AdvancedVacuumState) -> bool:
        # All rooms must be clean AND robot must be at charger
        all_clean = all(not dirt for row in state.dirt_grid for dirt in row)
        at_charger = state.position == self.charger_position
        return all_clean and at_charger

    def actions(self, state: AdvancedVacuumState) -> list:
        """Generate all possible actions from the current state."""
        actions = []
        row, col = state.position

        # Action 1: Charge (only at charger if battery < max)
        if state.position == self.charger_position and state.battery < self.max_battery:
            new_state = AdvancedVacuumState(
                state.dirt_grid,
                state.position,
                self.max_battery
            )
            actions.append(VacuumAction("Charge", new_state))

        # Action 2: Suck (if current position is dirty and has battery)
        if state.battery > 0 and state.dirt_grid[row][col]:
            new_grid = [row.copy() for row in state.dirt_grid]
            new_grid[row][col] = False
            new_state = AdvancedVacuumState(
                new_grid,
                state.position,
                state.battery - 1
            )
            actions.append(VacuumAction("Suck", new_state))

        # Action 3-6: Move Up/Down/Left/Right (if has battery and within bounds)
        if state.battery > 0:
            moves = [
                ("Up", -1, 0),
                ("Down", 1, 0),
                ("Left", 0, -1),
                ("Right", 0, 1)
            ]

            for direction, dr, dc in moves:
                new_row, new_col = row + dr, col + dc

                # Check bounds
                if 0 <= new_row < self.grid_size and 0 <= new_col < self.grid_size:
                    new_state = AdvancedVacuumState(
                        state.dirt_grid,
                        (new_row, new_col),
                        state.battery - 1
                    )
                    actions.append(VacuumAction(f"Move {direction}", new_state))

        return actions

    def heuristic(self, state: AdvancedVacuumState) -> int:
        """Heuristic function for A* search."""
        if self.heuristic_type == "none":
            return 0

        elif self.heuristic_type == "simple":
            # Count dirty rooms + distance to charger if not there
            dirty_count = sum(sum(row) for row in state.dirt_grid)

            if state.position != self.charger_position:
                # Manhattan distance to charger
                dist_to_charger = abs(state.position[0] - self.charger_position[0]) + \
                                 abs(state.position[1] - self.charger_position[1])
                return dirty_count + dist_to_charger

            return dirty_count

        elif self.heuristic_type == "advanced":
            # More sophisticated heuristic considering battery constraints
            dirty_count = sum(sum(row) for row in state.dirt_grid)

            if dirty_count == 0:
                # All clean, just need to get back to charger
                return abs(state.position[0] - self.charger_position[0]) + \
                       abs(state.position[1] - self.charger_position[1])

            # Estimate: dirty rooms + max distance in grid + return to charger
            # This is admissible as it underestimates
            max_dist = 6  # Max Manhattan distance in 4x4 grid
            return dirty_count + max_dist

        return 0


# Test the implementation
print("🤖 Testing Advanced Vacuum Problem:")
advanced_vacuum = AdvancedVacuumProblem()
print(f"\n🎯 Initial state:")
print(advanced_vacuum.initial_state())

print("\n🔍 Solving with BFS...")
result = breadth_first_search(advanced_vacuum)
print(result)

if result.found_solution and len(result.solution_path()) <= 20:
    print(f"\n🛤️ First 10 steps of solution:")
    for i, action in enumerate(result.solution_path()[:10], 1):
        print(f"{i:2d}. {action}")

# Compare all algorithms
print("\n" + "="*60)
print("🏁 Comparing all algorithms on Advanced Vacuum Problem:")
compare_algorithms(advanced_vacuum, "Advanced Vacuum (4x4 + Battery)")

print("\n💡 Testing with heuristics:")
print("\n" + "="*60)
print("Testing A* with SIMPLE heuristic:")
advanced_vacuum_h1 = AdvancedVacuumProblem(heuristic="simple")
result_h1 = a_star_search(advanced_vacuum_h1)
print(result_h1)

print("\n" + "="*60)
print("Testing A* with ADVANCED heuristic:")
advanced_vacuum_h2 = AdvancedVacuumProblem(heuristic="advanced")
result_h2 = a_star_search(advanced_vacuum_h2)
print(result_h2)

🤖 Testing Advanced Vacuum Problem:

🎯 Initial state:
🔋 Battery: 15/15
┌─────────────────┐
│ 🤖 💨 💨 💨 │
│ 💨 💨 💨 💨 │
│ 💨 💨 💨 💨 │
│ 💨 💨 💨 💨 │
└─────────────────┘

🔍 Solving with BFS...
✅ Solution found!
📊 Cost: 49
📏 Steps: 49
⏱️  Time: 462.5690s
🔍 Nodes expanded: 13492405
🛤️  Path: Suck → Charge → Move Down → Suck → Move Down → Suck → Move Down → Suck → Move Right → Suck → Move Up → Suck → Move Up → Suck → Move Up → Suck → Move Left → Charge → Move Down → Move Down...

🏁 Comparing all algorithms on Advanced Vacuum Problem:
🏁 Comparing algorithms on: Advanced Vacuum (4x4 + Battery)

Algorithm            Found?   Cost   Nodes    Time (s)  
------------------------------------------------------------
Breadth-First Search ✅ Yes    49     13492405 458.9294  
Depth-First Search   ✅ Yes    255    1617     0.0178    
Uniform Cost Search  ✅ Yes    49     13506720 657.8110  
A* Search            ✅ Yes    49     13506721 724.0917  

💡 Testing with heuristics:

Testing A* with SIMPLE heuristic:
✅ Solu

# Labyrinth problems

Challenge: Write the problem code (to be used with the same algorithms) to traverse a labyrinth.

This time, you need to do everything on your own!

In [18]:
print(get_problem('labyrinth-small'))  # Example API call to see the return format
print(get_problem('labyrinth-plus-big'))  # Example API call to see the return format

{'explanation': 'Natigate the mace, from S to E. # represents walls, . empty spaces. Allowed moves: U, D, L, R', 'final-pos': 'E', 'initial-pos': 'S', 'key-pos': 'K', 'map': '#####################\n#S......#.....#.#...#\n#.#.#.###.#.###.###.#\n#.#.#...#.#.#.....#.#\n###.#.#.#.#.#.#####.#\n#...#.#.#.#.#.......#\n###.###.#.###.#.###.#\n#.....#.......#.#...#\n#.###.#######.#######\n#...#.....#.#.......#\n#.#.#.#.###.#######.#\n#.#.#.#.......#.#.#.#\n#.#.#.###.#.#.#.#.###\n#.#.#.#...#.#.......#\n#.#.#.#####.#########\n#.#.#...#...........#\n#.#.#####.###.#####.#\n#.#...#.#...#.....#.#\n#.###.#.#.#.#.#######\n#...#.#...#.#......E#\n#####################', 'path-format': '[UDLR](-[UDLR])*'}
{'explanation': 'Natigate the mace, from S to E. # represents walls, . empty spaces. Allowed moves: U, D, L, R', 'final-pos': 'E', 'initial-pos': 'S', 'key-pos': 'K', 'map': '#####################################################################################################\n#S..........................

In [19]:
from collections import deque
import heapq
import itertools
import time
from typing import Optional

# ===============================
# State Classes
# ===============================

class State:
    pass

class Action:
    def __init__(self, name: str, result: State, cost: int = 1):
        self.name = name
        self.result = result
        self.resulting_state = result  # Add alias for compatibility
        self.cost = cost

    def __str__(self):
        return self.name


class LabyrinthState(State):
    def __init__(self, position: tuple[int, int], has_key: bool = False):
        self.position = position
        self.has_key = has_key

    def __eq__(self, other):
        return isinstance(other, LabyrinthState) and self.position == other.position and self.has_key == other.has_key

    def __hash__(self):
        return hash((self.position, self.has_key))

# ===============================
# Labyrinth Problem
# ===============================

class LabyrinthProblem:
    """Labyrinth maze with optional keys and heuristics."""

    def __init__(self, problem_data: dict):
        self.grid = [list(row) for row in problem_data['map'].strip().split('\n')]
        self.height = len(self.grid)
        self.width = len(self.grid[0]) if self.height > 0 else 0

        self.start = None
        self.goal = None
        self.key_positions = set()

        for r in range(self.height):
            for c in range(self.width):
                cell = self.grid[r][c]
                if cell == 'S':
                    self.start = (r, c)
                elif cell == 'E':
                    self.goal = (r, c)
                elif cell == 'K':
                    self.key_positions.add((r, c))

        self.needs_key = len(self.key_positions) > 0

    def initial_state(self) -> State:
        return LabyrinthState(self.start, has_key=False)

    def is_goal(self, state: LabyrinthState) -> bool:
        if state.position != self.goal:
            return False
        if self.needs_key and not state.has_key:
            return False
        return True

    def actions(self, state: LabyrinthState) -> list[Action]:
        actions = []
        row, col = state.position
        moves = [("Up", -1, 0), ("Down", 1, 0), ("Left", 0, -1), ("Right", 0, 1)]

        for name, dr, dc in moves:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < self.height and 0 <= new_col < self.width:
                if self.grid[new_row][new_col] != '#':
                    new_has_key = state.has_key or ((new_row, new_col) in self.key_positions)
                    new_state = LabyrinthState((new_row, new_col), new_has_key)
                    action = Action(name, new_state, cost=1)
                    actions.append(action)

        return actions

    # ===============================
    # Heuristics
    # ===============================

    def heuristic(self, state: LabyrinthState) -> int:
        if not self.needs_key:
            # Simple maze: Manhattan distance to goal
            return abs(state.position[0] - self.goal[0]) + abs(state.position[1] - self.goal[1])
        else:
            if state.has_key:
                # Key collected: distance to goal
                return abs(state.position[0] - self.goal[0]) + abs(state.position[1] - self.goal[1])
            else:
                # Distance to nearest key + distance from key to goal
                min_dist_to_key = min(abs(state.position[0]-k[0]) + abs(state.position[1]-k[1]) for k in self.key_positions)
                min_key_to_goal = min(abs(k[0]-self.goal[0]) + abs(k[1]-self.goal[1]) for k in self.key_positions)
                return min_dist_to_key + min_key_to_goal

    # ===============================
    # Maze display
    # ===============================

    def display_maze(self, path_positions=None):
        if path_positions is None:
            path_positions = set()
        print("\n🗺️  Maze:")
        for r in range(self.height):
            line = ""
            for c in range(self.width):
                pos = (r, c)
                if pos == self.start:
                    line += "S"
                elif pos == self.goal:
                    line += "E"
                elif pos in self.key_positions:
                    line += "K"
                elif pos in path_positions:
                    line += "."
                else:
                    line += self.grid[r][c]
            print(line)

# ===============================
# Solver
# ===============================

def solve_labyrinth(problem_data, algorithm="astar"):
    problem = LabyrinthProblem(problem_data)
    initial_state = problem.initial_state()

    # Select algorithm
    if algorithm == "bfs":
        result = breadth_first_search(problem)
    elif algorithm == "dfs":
        result = depth_first_search(problem)
    elif algorithm == "ucs":
        result = uniform_cost_search(problem)
    elif algorithm == "astar":
        result = a_star_search(problem)
    else:
        print(f"Unknown algorithm: {algorithm}")
        return

    print(f"\n--- {algorithm.upper()} Result ---")
    if result.found_solution:
        print(f"✅ Solution found! Steps: {result.steps}, Cost: {result.cost}, Nodes expanded: {result.nodes_expanded}")
        # Display path
        current_state = problem.initial_state()
        path_positions = {current_state.position}
        for action_name in result.solution_path():
            for action in problem.actions(current_state):
                if action.name == action_name:
                    current_state = action.result
                    path_positions.add(current_state.position)
                    break
        problem.display_maze(path_positions)
    else:
        print("❌ No solution found.")

# ===============================
# Example usage
# ===============================

# Load problem data
labyrinth_small = get_problem('labyrinth-small')
labyrinth_big = get_problem('labyrinth-big')
labyrinth_plus_small = get_problem('labyrinth-plus-small')
labyrinth_plus_big = get_problem('labyrinth-plus-big')
labyrinth_massive = get_problem('labyrinth-massive')

# Compare algorithms on each maze
compare_algorithms(LabyrinthProblem(labyrinth_small), "Labyrinth Small")
compare_algorithms(LabyrinthProblem(labyrinth_big), "Labyrinth Big")
compare_algorithms(LabyrinthProblem(labyrinth_plus_small), "Labyrinth Plus Small (Key)")
compare_algorithms(LabyrinthProblem(labyrinth_plus_big), "Labyrinth Plus Big (Key)")
compare_algorithms(LabyrinthProblem(labyrinth_massive), "Labyrinth Massive")

🏁 Comparing algorithms on: Labyrinth Small

Algorithm            Found?   Cost   Nodes    Time (s)  
------------------------------------------------------------
Breadth-First Search ✅ Yes    36     198      0.0011    
Depth-First Search   ✅ Yes    36     144      0.0007    
Uniform Cost Search  ✅ Yes    36     198      0.0013    
A* Search            ✅ Yes    36     117      0.0011    
🏁 Comparing algorithms on: Labyrinth Big

Algorithm            Found?   Cost   Nodes    Time (s)  
------------------------------------------------------------
Breadth-First Search ✅ Yes    200    4998     0.0274    
Depth-First Search   ✅ Yes    200    2334     0.0115    
Uniform Cost Search  ✅ Yes    200    4998     0.0325    
A* Search            ✅ Yes    200    1667     0.0162    
🏁 Comparing algorithms on: Labyrinth Plus Small (Key)

Algorithm            Found?   Cost   Nodes    Time (s)  
------------------------------------------------------------
Breadth-First Search ✅ Yes    36     204      0.0